In [1]:
from IPython import get_ipython
ip = get_ipython()
ip.run_line_magic("reload_ext", "autoreload")  # these will enable module autoreloading
ip.run_line_magic("autoreload", "2")

### Get List of Reports Available

In [7]:
import os
from glob import glob
import ExportProvider.IBRK.Extract as ibrk
from lxml import etree
from ReportingStrategies.Slovenia.ReportGeneration import ReportGenerator
from ReportingStrategies.Slovenia.ReportGeneration import DividendReport
import ReportingStrategies.Slovenia.Mappers.IBRK as ibrkMapper
from Common.Configuration import TaxPayerInfo
from Common.Configuration import TaxPayerType
from Common.Configuration import ReportBaseConfig
from arrow import Arrow
import arrow

rootOfProject = os.getcwd()

reportsDirectory = os.path.join(rootOfProject, "BrokerExports")
generatedDirectory = os.path.join(rootOfProject, "GeneratedReports")

brokerExports = glob("*.xml", root_dir=reportsDirectory)
print(brokerExports)


def loadFileAndExtractLines(file: str):
    fileLocation = os.path.join(reportsDirectory, file)
    
    with open(fileLocation) as fobj:
        xml = fobj.read()

    root = etree.fromstring(xml)
    transactions = ibrk.extractCashTransactionFromXML(root)
    return transactions

brokerReports = list(map(loadFileAndExtractLines, brokerExports))
mergedReports = ibrk.mergeCashTransactions(brokerReports)

taxPayerInfo = TaxPayerInfo(
    taxNumber = "davcna",
    taxpayerType = TaxPayerType.PHYSICAL_SUBJECT,
    name = "ime priimek",
    address1 = "naslov",
    address2 = None,
    city = "mesto",
    postNumber = "postna #",
    postName = "posta",
    municipalityName = "obcina",
    birthDate = Arrow.now(),
    maticnaStevilka = "emsa",
    invalidskoPodjetje = False,
    resident = True,
    activityCode = "",
    activityName = "",
    countryID = "SI",
    countryName = "Slovenia"
)
basicReport = ReportGenerator(taxPayerInfo)
envelope = basicReport.createReportEnvelope()


reportconfig = ReportBaseConfig(fromDate=arrow.get("2022"), toDate=arrow.get("2023"))
dividendReport = DividendReport(reportConfig=reportconfig, taxPayerInfo=taxPayerInfo)

convertedCommonFormat = ibrkMapper.getExportGenericDividendLineFromCashTransactions(mergedReports)

csvReport = dividendReport.generateDataFrameReport(convertedCommonFormat)


csvReport.to_csv(os.path.join(generatedDirectory, 'export.csv'), index=False)



['Tax_Relevant_Export_2022.xml']
Failed online lookup of ISIN (US9663875089), falling back to hardcoded company definitions


## csv to XML

In [8]:
from ReportingStrategies.Slovenia.CommonReport import CommonReport
from ReportingStrategies.Slovenia.CommonReport import TaxPayerInfo
from ReportingStrategies.Slovenia.CommonReport import TaxPayerType
from arrow import Arrow
from ReportingStrategies.Slovenia.DividendsReport import DividendsReport
from ReportingStrategies.Slovenia.DividendsReport import DividendReportConfig
from lxml import etree

taxPayerInfo = TaxPayerInfo(
    taxNumber="davčna",
    taxpayerType=TaxPayerType.PHYSICAL_SUBJECT,
    name="ime",
    address1="naslov",
    address2=None,
    city="mesto",
    postNumber="poštna št.",
    postName="posta",
    municipalityName="občina",
    birthDate=Arrow.now(),
    maticnaStevilka="EMSA",
    invalidskoPodjetje=False,
    resident=True,
    activityCode="ne vem",
    activityName="ne vem",
    countryID="SI",
    countryName="Slovenia"
)

commonReport = CommonReport(taxPayerInfo)

base = commonReport.createReportEnvelope()


dividendLines = reported.to_dict("records")
generatedReportXml = dividendReport.appendToXMLForSubmission(dividendLines, DividendReportConfig("2022"), taxPayerInfo, base)



generatedDirectory = os.path.join(rootOfProject, "GeneratedReports")

etree.ElementTree(generatedReportXml).write(os.path.join(generatedDirectory, "Doh_Div_3-gen.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)


In [4]:
from lxml import etree
ns_edp = "http://edavki.durs.si/Documents/Schemas/EDP-Common-1.xsd"

nsmap = {
    None: "http://edavki.durs.si/Documents/Schemas/Doh_Div_3.xsd",
    "edp": ns_edp,
    "xs": "http://www.w3.org/2001/XMLSchema"
}

Envelope = etree.Element("Envelope", {}, nsmap=nsmap)

Header = etree.SubElement(Envelope, etree.QName(ns_edp, "Header"))
Taxpayer = etree.SubElement(Header, etree.QName(ns_edp,"taxpayer"))
etree.SubElement(Taxpayer, etree.QName(ns_edp, "taxNumber")).text = "davčna"
etree.SubElement(Taxpayer, etree.QName(ns_edp, "taxpayerType")).text = "FO" # see EDP-Common-1 schema
etree.SubElement(Taxpayer, etree.QName(ns_edp, "name")).text = "ime priimek"
etree.SubElement(Taxpayer, etree.QName(ns_edp, "address1")).text = "naslov"
etree.SubElement(Taxpayer, etree.QName(ns_edp, "address2")).text = ""
etree.SubElement(Taxpayer, etree.QName(ns_edp, "city")).text = "mesto"
etree.SubElement(Taxpayer, etree.QName(ns_edp, "postNumber")).text = "pošna številka"
etree.SubElement(Taxpayer, etree.QName(ns_edp, "postName")).text = "pošta"
etree.SubElement(Envelope, etree.QName(ns_edp, "AttachmentList"))
etree.SubElement(Envelope, etree.QName(ns_edp, "Signatures"))

workflow = etree.SubElement(Header, etree.QName(ns_edp, "Workflow"))
etree.SubElement(workflow, etree.QName(ns_edp, "DocumentWorkflowID")).text = "O" # https://edavki.durs.si/EdavkiPortal/PersonalPortal/[360253]/Pages/Help/sl/WorkflowType1.htm

body = etree.SubElement(Envelope, "body")
# etree.SubElement(body, etree.QName(ns_edp, "bodyContent"))

Doh_Div = etree.SubElement(body, "Doh_Div")
etree.SubElement(Doh_Div, "Period").text = "leto"
etree.SubElement(Doh_Div, "EmailAddress").text = "email"
etree.SubElement(Doh_Div, "PhoneNumber").text = "telefonska"
etree.SubElement(Doh_Div, "ResidentCountry").text = "SI"
etree.SubElement(Doh_Div, "IsResident").text = "true"
# etree.SubElement(Doh_Obr, "Locked").text = False
# etree.SubElement(Doh_Obr, "SelfReport").text = False
# etree.SubElement(Doh_Obr, "WfTypeU").text = False

def transformDividendLineToXML(line: dict[str, str]):
    dividendLine = etree.SubElement(body, "Dividend")
    etree.SubElement(dividendLine, "Date").text = line['Datum prejema dividend']
    etree.SubElement(dividendLine, "PayerTaxNumber").text = line['Davčna številka izplačevalca dividend']
    etree.SubElement(dividendLine, "PayerIdentificationNumber").text = line['Identifikacijska številka izplačevalca dividend']
    etree.SubElement(dividendLine, "PayerName").text = line['Naziv izplačevalca dividend']
    etree.SubElement(dividendLine, "PayerAddress").text = line['Naslov izplačevalca dividend']
    etree.SubElement(dividendLine, "PayerCountry").text = line['Država izplačevalca dividend']
    etree.SubElement(dividendLine, "Type").text = line['Šifra vrste dividend']
    etree.SubElement(dividendLine, "Value").text = str(line['Znesek dividend (v EUR)'])
    etree.SubElement(dividendLine, "ForeignTax").text = str(line['Tuji davek (v EUR)'])
    etree.SubElement(dividendLine, "SourceCountry").text = line['Država vira']
    etree.SubElement(dividendLine, "ReliefStatement").text = line['Uveljavljam oprostitev po mednarodni pogodbi']

dividendLines = reported.to_dict("records")
for line in dividendLines:
    transformDividendLineToXML(line)    # type: ignore

generatedDirectory = os.path.join(rootOfProject, "GeneratedReports")

etree.ElementTree(Envelope).write(os.path.join(generatedDirectory, "Doh_Div_3.xml"), xml_declaration=True, encoding='utf-8', pretty_print=True)
